Run the following command:

pip install -r requirements.txt

In [ ]:
import re
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential,load_model
#from tensorflow.python.keras.utils import _____
from keras.layers import Embedding,LSTM,Dense,Dropout
from sklearn.model_selection import train_test_split

## Importing all the relevant libraries

In [ ]:
from dotenv import load_dotenv, dotenv_values
secrets = dotenv_values(".env")

#Imports the credentials for Twitter

consumerKey = secrets["consumerKey"]
consumerSecret = secrets["consumerSecret"]
accessToken = secrets["accessToken"]
accessSecret = secrets["accessSecret"]

In [ ]:
authenticate = tweepy.OAuthHandler(consumerKey,consumerSecret)
authenticate.set_access_token(accessToken,accessSecret)

api = tweepy.API(authenticate,wait_on_rate_limit= True)

In [ ]:
mention = api.mentions_timeline()
print(mention)

for tweets in mention:
  print(str(tweets.id)+'-'+tweets.text)

In [ ]:
dftweet = pd.DataFrame(['I hate him'],columns = ["Column2"])
dftweet.head()

In [ ]:
#clean tweets
def cleanTxt(text):
  text =re.sub(r'@[A-Za-z0-9]+','',text) #remove @
  text = re.sub(r'#','',text)# remove #
  text = re.sub(r'RT[\s]+','',text)# remove RT
  text = re.sub(r'https?:\/\/\S+','',text)#remove hyperlink

  return text

dftweet['Column2']= dftweet['Column2'].apply(cleanTxt)
print(dftweet)

In [ ]:
df = pd.read_csv('./datasets/combined.csv')
df = pd.read_csv('./datasets/new.csv')
df.Column1 =df.Column1.astype(str)
df.Column2 =df.Column2.astype(str)
df['Column1']= df['Column1'].apply(cleanTxt)
df.head()
#df.shape

In [ ]:
indexNames = df[ df['Column2'] == 'nan' ].index
df.drop(indexNames , inplace=True)
df['Column2'].value_counts()

In [ ]:

tokenizer = Tokenizer(num_words=5000,split =" ")
tokenizer.fit_on_texts(df['Column1'].values)

df_x = tokenizer.texts_to_sequences(df['Column1'].values)
df_x = pad_sequences(df_x)
print(df_x[:5])

tokenizer.fit_on_texts(dftweet['Column2'].values)
x_tweet = tokenizer.texts_to_sequences(dftweet['Column2'].values)
x_tweet = pad_sequences(x_tweet)

print(df_x.shape)

In [ ]:
df_y = pd.get_dummies(df['Column2']).values
[print(df['Column2'][i],df_y[i]) for i in range(101,135)]
print(df_x.shape)
print(df_y.shape)
x_train,x_test,y_train,y_test = train_test_split(df_x,df_y,test_size = 0.2,random_state = 0)

In [ ]:
model = Sequential()
model.add(Embedding(5000,256,input_length=df_x.shape[1]))
#model.add(Dense(4, activation='softmax'))
model.add(Dropout(0.3))
model.add(LSTM(256,return_sequences=True,dropout=0.3,recurrent_dropout=0.2))
model.add(LSTM(256,dropout=0.3,recurrent_dropout=0.2))
model.add(Dense(6,activation='softmax'))

In [25]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [ ]:
model.fit(x_train, y_train,epochs=10,batch_size =32,verbose=2)

In [ ]:
model.save('sentiment')

In [ ]:
# LOOKS AT THE CONTENT OF THE MENTIONED TWEET
#tweets = api.mentions_timeline(tweet_mode='extended')
#a = tweets[3].in_reply_to_status_id_str
#tweet_a = api.get_status(id=a)
#print(tweet_a.text)
def check (i):
    print(i)
    print(np.max(i))
    a = np.where(i == np.max(i))
    a = a[0][0]
    if ( a == 0):
        return('Anger')
    elif (a == 1):
        return('Fear')
    elif (a ==2):
        return('Joy')
    elif (a ==3):
        return('Love')
    elif (a ==4):
        return('Sadness')   
    else:
        return('Surprised')

predictions = model.predict(x_tweet)
print (dftweet.Column2[0])
#print ( (np.where(predictions[0] == np.max(predictions[0])))[0][0])
#[print(df['Column1'][i],predictions[i],y_test[i]) for i in range (0,1)]
#print(check(y_test[0]))
print(check(predictions[0]))